# Notebook 7 - Crear y consultar una Base de Datos no relacional orientada a Documentos (Mongo)

## Ejercicio 1: Primeros pasos desde Mongo CLI

En un terminal, verificar si el SGBD Mongo ya está iniciado:

<code>ps -aux | grep mongod</code>

Si no está iniciado, iniciarlo con el comando: <code>mongod</code>

Luego, abrir otra pestaña de terminal (CTRL + Shift + T) e iniciar el cliente CLI de mongo: <code>mongo</code>.

1) Creamos una nueva base de datos llamado <b>nb7-peliculas</b>

<code>use nb7-peliculas</code>

Deberian ver como respuesta del sistema la respuesta: "switched to db nb7-peliculas"

Mongo CLI no utiliza SQL para realizar operaciones CRUD (Create/Read/Update/Delete), utiliza su propia API parecida al lenguaje Javascript.

La variable <code>db</code> representa siempre la base de datos que estamos utilizando (en este caso "nb7-peliculas").

2) Creamos una colección <b>actors</b>

<code>db.actors.insert({firstname:"Will", lastname:"Smith"})</code>

Como pueden verlo, para crear la colección no tuvimos que definir la estructura de la colección, simplemente hemos insertado un documento.

Verificamos que hemos realmente creado la coleccion con:

<code>db.actors.find()</code>

3) Insertamos varios documentos en la colección en un solo comando:

<code>db.actors.insertMany([{firstname:"Scarlett", lastname:"Johanson"},{firstname:"Matt", lastname:"Damon"},{firstname:"Nathan", lastname:"Smith"}])</code>

4) Contamos cuántos documentos tenemos en la colección:

<code>db.actors.count()</code>

Como en SQL, la API de Mongo permite construir consultas para filtrar, ordenar o agrupar datos. Sin embargo, la síntaxis es distinta (y amenudo menos simple e intuitiva que SQL).

5) Filtramos los actores que se llaman 'Smith'

<code>db.actors.find({lastname:"Smith"})</code>

Mongo tambien permite modificar los datos (ojo: Mongo no tiene mecanismo interno para asegurar la coherencia de los datos, es la responsabilidad del programador de la aplicación).

6) Actualizamos un documento. Por ejemplo, "Nathan Smith" se llama ahora "Nathan Lane"

<code>db.actors.update({firstname:"Nathan", lastname:"Smith"},{$set : {lastname:"Lane"}})</code>

Este último comando va a actualizar el primer documento encontrado que corresponde al filtro definido.

7) Supongamos ahora que queremos agregar un campo "gender" en todos los documentos:

¿Qué hace el comando siguiente? ¿Cuál es el limite?

<code>db.actors.update({},{$set : {gender:"M"}})</code>

Para actualizar todos los documentos, y no simplemente el primero, podemos utilizar el comando:

<code>db.actors.update({},{$set : {gender:"M"}},{multi:true})</code>

8) Actualizar el documento "Scarlett Johanson", para indicar que es una mujer (gender:"F").

9) Para suprimir un atributo, podemos utilizar el operador $unset. Por ejemplo:
    
<code>db.actors.update({},{$unset : {gender:1}},{multi:true})</code>

10) Para suprimir un documento completo, podemos utilizar el comando:

<code>db.actors.remove({lastname:"Smith"})</code>

11) Agregamos un atributo de tipo arreglo en un documento

<code>db.actors.update({firstname:"Matt"},{$set : {tab:[1,"a"]}})</code>

12) Añadimos un valor en el arreglo con el operator $push

<code>db.actors.update({firstname:"Matt"},{$push : {tab:["new_value"]}})</code>

¿Cuál es el problema con el último comando? ¿Cuál es la diferencia con el comando siguiente?

<code>db.actors.update({firstname:"Matt"},{$push : {tab:"new_value"}})</code>

Cómo pueden verlo el SGBD no relacional Mongo es mucho más flexible que MySQL, pero deja mucho más posibilidad de equivocarse.


No duden en consultar la documentación de Mongo para probar otros comandos CRUD: https://docs.mongodb.com/manual/crud/


## Ejercicio 2: Primeras consultas avanzadas con una colección de peliculas

En este segundo ejercicio, trabajaremos con una base de datos existente llamada "movie details", la cual tiene 2295 documentos JSON resumiendo información sobre peliculas.

- Descargar el dataset https://raw.githubusercontent.com/steveren/docs-assets/charts-tutorial/movieDetails.json. ¿Qué contiene este archivo? Un documento JSON? Un arreglo de documentos JSON? Una lista de documentos JSON?

- Importamos los datos en Mongo con el comando siguiente:

<code>mongoimport --db nb7-peliculas --collection movieDetails --drop --file <path/to/movieDetails.json></code>

A continuación, queremos realizar algunas consultas típicas para observar la diferencia entre el lenguaje de consulta SQL y el lenguaje de Mongo.

<b>Nota bena</b>: ¿Qué pasa si agregan la función <code>.pretty()</code> al final de un comando? Por ejemplo: <code>db.actors.find().pretty()</code>

Si agregamos .pretty al final del comando, la respuesta enviada por la bd es ordenada en una tabla



1) Utilizar un comando Mongo para contar cuántos documentos hay en la colección 'movieDetails'. ¿Cuál sería el equivalente en SQL?

db.movieDetails.count()

SELECT COUNT(*) FROM movieDetails

2) Mostrar los títulos (title) y años de producción (year) de todas las peliculas de la colección. ¿A qué clausula SQL correspondría?


db.movieDetails.find({},{title:1,year:1})

SELECT title, year FROM movieDetails </code>

3) Contar las peliculas realizadas entre 1990 y 2000


db.movieDetails.count({$and:[{year:{$gt:1990}},{year:{$lt:2000}}]})

SELECT COUNT(total) from (SELECT * as total from movieDetails WHERE year BETWEEN "1990" and "2000") as t2 </code>

4) ¿Cuáles son los titulos de peliculas que ganaron más de 5 awards?


db.movieDetails.find({"awards.wins":{$gt:5}},{title:1,"awards.wins":1})

5) ¿Cuál es el rating promedio en el sitio imdb para todas las peliculas?


db.movieDetails.aggregate([{$group: {_id:null, promedio: {$avg:"$imdb.rating"} } }])

6) ¿Cuáles son las peliculas en las cuales participó el actor Nicolas Cage?


db.movieDetails.find({actors:"Nicolas Cage"},{title:1})

7) ¿Cuál es el número de pelicula por año?


db.movieDetails.aggregate([{$group:{_id:"$year",sumapeliculas:{$sum:1}}}])

8) ¿Cuál es la pelicula con el mejor rating?


db.movieDetails.find({},{title:1,_id:0}).sort({"imdb.rating":-1}).limit(1)

9) ¿Cuáles son los thrillers con un rating de al meno 6?


db.movieDetails.find(
    {
        $and:
        [
            {"imdb.rating":{$gt:6}},
            {genres:"Thriller"}

    ]
},
{title:1,_id:0,"imdb.rating":1}

) </code>

10) ¿Cuáles son todos los valores de generos posibles?


db.movieDetails.distinct( "genres" )

11) ¿Cuáles son las peliculas que contienen la palabra 'Star'?


db.movieDetails.find({"title": {$regex:/Star /,$options:"i"}},{title:1,_id:0})

12) ¿Cuáles son los títulos que empiezan con la letra 'T'?


db.movieDetails.find({"title": {$regex:/^T/}},{title:1,_id:0})

13) ¿Cuáles son las peliculas que no son de tipo Drama o Thriller?


db.movieDetails.find(
    {
        $and:
        [
            {genres:{$ne:"Drama"}},
            {genres:{$ne:"Thriller"}}

    ]
},
{title:1,_id:0}

) 

## Ejercicio 3: Integrar Mongo con Python

Desde un entorno Python, conectarse a su base de datos movieDetails y realizar consultas similares al ejercicio 2.

In [3]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["nb7-peliculas"]
mycol = mydb["movieDetails"]

In [4]:
#1), ejercicio 2
mydoc = mydb.movieDetails.estimated_document_count()

In [5]:
#2), ejercicio2
mydoc = mydb.movieDetails.find({},{'title':1,'year':1,'_id':0}).limit(20)

In [6]:
#3), ejercicio2
mydoc = mydb.movieDetails.count_documents({'$and':[{'year':{'$gt':1990}},{'year':{'$lt':2000}}]})

## Ejercicio 4: Conceptos avanzados de Base de Datos: MySQL y Mongo

- <b>Indíce: optimización de consultas</b>

Referencias:
- Indíces en Mongo: https://docs.mongodb.com/manual/indexes/
- Indíces en MySQL: https://dev.mysql.com/doc/refman/8.0/en/optimization-indexes.html
- Introducción a indíces en MySQL: https://www.adictosaltrabajo.com/2015/09/11/introduccion-a-indices-en-mysql/

1) ¿Qué es un Indíce y de qué sirve?

2) ¿Cuándo los SGBDs utilizan los Indíces?

3) ¿Cuáles son los tipos de Indíces en MySQL y Mongo?

4) Si los indíces son utiles, ¿por qué no crear indíces por cada columna/atributo? ¿Cuáles son los límimtes?

5) Conectarse a la base de datos 'sakila' de MySQL y ejecutar la consulta siguiente para poder conservar información sobre el rendimiento de consultas:

<code>set profiling=1;</code>

Ejecutar 10 veces la consulta siguiente:
<code>select * from rental where return_date>="2005-08-25 17:48:44";</code>

Luego, ejecutar el comando:
<code>show profiles;</code>

¿Cuál es la duración promedia del SGBD para responder a esta consulta?

6) Crear un index sobre la columna 'return_date', y medir nuevamente 10 veces el rendimiento de la misma consulta. ¿Cuál es el nuevo valor promedio?

7) ¿Cómo crear índices en Mongo?

- <b>Transacciones</b>

Referencias:
- Introducción a Transacciones en MySQL: http://raknarrok.blogspot.com/2011/03/introduccion-transacciones.html
- Transacción en MySQL: https://dev.mysql.com/doc/refman/8.0/en/commit.html
- Transacción en Mongo: https://docs.mongodb.com/manual/core/transactions/

1) ¿Qué es una transacción? 

2) ¿En términos de transacción, cuáles son las características que deberian garantizar un buen SGBD?

3) En MySQL, crear una tabla <b>bank_user(name VARCHAR(10), account INT)</b> que permite conocer el monto disponible en la cuenta bancaria de algunos usuarios.

4) Realizar una operación que permita transferir 100.000 pesos del usuario A al usuario B. ¿Cómo garantizar que la operación no va a generar algún incoherencia en los datos? Escribir el código SQL correspondiente.

5) Supongamos que queremos hacerlo mismo en Mongo. ¿Mongo soporta transacciones ACID? ¿Desde cuándo? ¿Cómo se puede hacer transacciones? ¿Con qué limitaciones?


- <b>Sharding y Replicación</b>

1) ¿Cuál es el principio del Sharding?

2) ¿Cuál es el principio de la Replicación?

3) ¿MySQL implementa mecanismos de Sharding/Replicación? y Mongo?

índice 

1) Un indice genera punteros a las posiciones en memoria de las tablas de la base de datos, sirve para acelerar los tiempos de respuesta en consultas de tipo SELECT, correspondientes a sql.
2) Cuando se requieren hacer consultas de datos, buscar una tabla, su relacion con otras tablas.
3) MYSQL: INDEX (NON-UNIQUE | UNIQUE | PRIMARY | FULLTEXT)
4) Depende de los requisitos de las consultas, ya que generar indices ocupa espacio en disco.
5) set profiling=1; (0.00503925+0.00508350+0.005111275+0.00508550+0.00512650+0.00507550+0.00508100+0.00510925+0.00507700+0.00514400)/10 = 0.0050932775 [s]
6) ALTER TABLE rental ADD INDEX (return_date);

Transacciones

1) Una transaccion es un conjunto de operaciones en una base de datos, estas que se ejecutan todas (o ninguna si ocurriese un error).
2) Una trasaccion debe ser de tipo ACID, debe cumplir con:

Atomicidad: o se realizan todas las operaciones de la transaccion o no se realiza ninguna
Consistencia: las transacciones deben realizarse de forma aislada (no puede existir concurrencia sobre los mismos datos)
Aislamiento (Isolation): Una operacion no puede afectar otra que se ejecuta de forma concurrente.
Durabilidad: Una vez ejecutada una operacion, esta debe persistir en el tiempo.

3)create database bank;
use bank;
create table bank_user(name VARCHAR(10), account INT);
insert into bank_user values ("user1", 11000000);
insert into bank_user values ("user2", 30000);
4)
//Ejecutar la transaccion
begin;
update bank_user set account = (account + 100000) where name="juanito";
update bank_user set account = (account - 100000) where name="pedrito";
//Confirmar cambios
commit;
5)Es posible, Mongo las soporta desde la version 4.0.
Se pueden realizar con los comandos:
.startTransaction() //Inicia una transaccion
.commitTransaction() //Para hacer confirmar
.abortTransaction() //Para cancelar
Limitaciones: No es posible realizar transacciones que afecten a catalogo de la base de datos, como crear o eliminar una coleccion.

Sharding y replicación

1)Consiste en particionar los datos de la base de datos, horizontalmente para lograr un redireccionaiento mas rapido de datos.
2)Consiste en realizar copias exactas de la base de datos, para tener la informacion en varias ubicaciones.
3)El sharding no esta implementado de forma directa por mysql, aunque si es posible usando MySql Cluster.
En Mysql la replicacion si es posible, utilizando el sistema Master/Slave, donde las modificaciones realizadas en un servidor "maestro" son replicadas en un servidor "esclavo".
Como mongo fue diseñado para sistemas donde se maneja mucha informacion, la distribucion horizontal de los datos es la base de su funcionamiento, el sharding y la replicacion esta integrado.